# Control Reachy's gaze in Unity using the look_at function

This notebook shows you how to control Reachy's head orientation by using the look_at function.

The look_at function takes a few argument:
* (x, y, z) the 3D coordinates of the point you want to look at (in Reachy's coordinate system : x is forward, y is left and z s up)
* duration: head motion duration to go from its current orientation to the requested one (in sec.)
* wait (bool): whether or not to wait for the end of the movement

Below, we illustrate how to use it inside a loop to let you define new look target using slider widgets.

In [ ]:
from reachy import parts, Reachy

reachy = Reachy(
    left_arm=parts.LeftArm(io='ws', hand='force_gripper'),
    right_arm=parts.RightArm(io='ws', hand='force_gripper'),
    head=parts.Head(io='ws'),    
)

You can now connect your robot in Unity.

In [ ]:
from IPython.display import clear_output


class LookAt(object):
    def __init__(self, head, init_target=(0.5, 0, 0), period=0.05):
        self.head = head
        
        self.trajs = None
        
        self.target = init_target
        self.period = period
        
    def set_target(self, x, y, z):
        if self.target == (x, y, z):
            return
        
        if self.trajs is not None:
            for t in self.trajs:
                t.stop()
                
        self.target = (x, y, z)
        try:
            self.trajs = self.head.look_at(x, y, z, duration=self.period, wait=False)
        except (ValueError, ZeroDivisionError):
            clear_output(wait=True)
            print(f'target {self.target} out of reach!')

In [ ]:
look_at = LookAt(reachy.head)

In [ ]:
from ipywidgets import interact

def loop(x, y, z):
    look_at.set_target(x, y, z)
    
_ = interact(
    loop,
    x=0.5,
    y=(-1, 1, 0.01),
    z=(-1, 1, 0.01),
)